In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import markdownify
import pandas as pd

In [2]:
url = "<<url-link>>"

# Open the url file
file = urlopen(url)
filebytes = file.read()

# get full html of the url with BeautifulSoup
html = BeautifulSoup(filebytes, features="html.parser")
for script in html(["script"]):
    script.extract()

file.close()

In [3]:
# convert html to markdown
md = markdownify.markdownify(str(html))

# text to dataframe
df = pd.DataFrame(md.splitlines())

In [4]:
# find all ``` syntax indexes
codeblock = df.loc[df[0] == "```"]
indexlist = codeblock.index

# get all code snippets and store them in a list
codesnippet = []
for index in range(0, len(indexlist), 2):
    start = indexlist[index] + 1
    stop = indexlist[index+1]
    codesnippet.append(df[start:stop])

In [5]:
# default output filename: last id of url path
filename = url.split('/')[-1]
fileextension = '.cs'
filepath = 'output/' + filename + fileextension

outfile = open(filepath, 'w')

for index, code in enumerate(codesnippet):
    codetext = code[0].str.cat(sep='\n')
    
    if index != 0: outfile.write("\n")
    outfile.write("%s" % codetext)
    
outfile.close()

In [6]:
df.to_csv('output/' + filename + '.csv')